This notebook aims to create a plot showing the effect of each transformation method on expressivity and accuracy

## Dataset

In [1]:
!pip install ogb

  Using cached outdated-0.2.2-py2.py3-none-any.whl.metadata (4.7 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 482.4 kB/s eta 0:00:00a 0:00:01
Using cached outdated-0.2.2-py2.py3-none-any.whl (7.5 kB)


In [3]:
from ogb.graphproppred import PygGraphPropPredDataset

dataset = PygGraphPropPredDataset(
    name='ogbg-molhiv', root='./dataset/OGB')

In [2]:
dataset[0]

Data(edge_index=[2, 40], edge_attr=[40, 3], x=[19, 9], y=[1, 1], num_nodes=19)

In [5]:
!pip install matplotlib

  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.7/164.7 kB 1.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 961.3 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 271.5/271.5 kB 703.8 kB/s eta 0:00:00a 0:00:01
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 1.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 1.6 MB/s eta 0:00:00


In [6]:
import matplotlib.pyplot as plt
import networkx as nx
from torch_geometric.utils import to_networkx

plt.figure(figsize=(100,100))

for i, data in enumerate(dataset[0:8], 1):
    plt.subplot(4, 2, i)
    G = to_networkx(data)
    nx.draw(G, pos=nx.spring_layout(G), with_labels=True, node_size=2000, font_size=30, edge_color="gray")

plt.show()

Matplotlib is building the font cache; this may take a moment.


In [1]:
import torch
from torch_geometric.loader import DataLoader
import torch.optim as optim
import torch.nn.functional as F

from tqdm import tqdm
import argparse
import time
import numpy as np

### importing OGB
from ogb.graphproppred import PygGraphPropPredDataset, Evaluator

cls_criterion = torch.nn.BCEWithLogitsLoss()
reg_criterion = torch.nn.MSELoss()

def train(model, device, loader, optimizer, task_type):
    model.train()

    for step, batch in enumerate(tqdm(loader, desc="Iteration")):
        batch = batch.to(device)

        if batch.x.shape[0] == 1 or batch.batch[-1] == 0:
            pass
        else:
            pred = model(batch)
            optimizer.zero_grad()
            ## ignore nan targets (unlabeled) when computing training loss.
            is_labeled = batch.y == batch.y
            if "classification" in task_type: 
                loss = cls_criterion(pred.to(torch.float32)[is_labeled], batch.y.to(torch.float32)[is_labeled])
            else:
                loss = reg_criterion(pred.to(torch.float32)[is_labeled], batch.y.to(torch.float32)[is_labeled])
            loss.backward()
            optimizer.step()

def eval(model, device, loader, evaluator):
    model.eval()
    y_true = []
    y_pred = []

    for step, batch in enumerate(tqdm(loader, desc="Iteration")):
        batch = batch.to(device)

        if batch.x.shape[0] == 1:
            pass
        else:
            with torch.no_grad():
                pred = model(batch)

            y_true.append(batch.y.view(pred.shape).detach().cpu())
            y_pred.append(pred.detach().cpu())

    y_true = torch.cat(y_true, dim = 0).numpy()
    y_pred = torch.cat(y_pred, dim = 0).numpy()

    input_dict = {"y_true": y_true, "y_pred": y_pred}

    return evaluator.eval(input_dict)


In [15]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GINEConv, global_add_pool
from torch.nn import Sequential, Linear, ReLU, BatchNorm1d

class GINENet(torch.nn.Module):
    def __init__(self, num_node_features, dim_h, edge_attr):
        super(GINENet, self).__init__()
        
        # Define GINE layers with the specified edge_dim
        self.conv1 = GINEConv(
            Sequential(Linear(num_node_features, dim_h),
                       BatchNorm1d(dim_h), ReLU(),
                       Linear(dim_h, dim_h), ReLU()),
            edge_dim=edge_attr)
        self.conv2 = GINEConv(
            Sequential(Linear(dim_h, dim_h), BatchNorm1d(dim_h), ReLU(),
                       Linear(dim_h, dim_h), ReLU()),
            edge_dim=edge_attr)
        self.conv3 = GINEConv(
            Sequential(Linear(dim_h, dim_h), BatchNorm1d(dim_h), ReLU(),
                       Linear(dim_h, dim_h), ReLU()),
            edge_dim=edge_attr)
        
        # Define linear layers for classification or regression
        self.lin1 = Linear(dim_h * 3, dim_h * 3)
        self.lin2 = Linear(dim_h * 3, 1)

    def forward(self, data):
        x, edge_index, edge_attr, batch = data.x, data.edge_index, data.edge_attr, data.batch
        
        # Convert node features to float32 if they are not already
        x = x.to(torch.float32)
        edge_attr = edge_attr.to(torch.float32)
        
        # Pass node features and edge attributes through GINE layers
        h1 = self.conv1(x, edge_index, edge_attr)
        h2 = self.conv2(h1, edge_index, edge_attr)
        h3 = self.conv3(h2, edge_index, edge_attr)

        # Apply global pooling for graph-level output
        h1 = global_add_pool(h1, batch)
        h2 = global_add_pool(h2, batch)
        h3 = global_add_pool(h3, batch)

        # Concatenate pooled features and pass through final linear layers
        h = torch.cat((h1, h2, h3), dim=1)
        h = self.lin1(h).relu()
        h = F.dropout(h, p=0.5, training=self.training)
        h = self.lin2(h)
        return h

In [16]:
def main():
    # Training settings
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    ### automatic dataloading and splitting
    dataset = PygGraphPropPredDataset(name='ogbg-molhiv', root='./dataset/OGB')

    file_name = "OGB_test"


    split_idx = dataset.get_idx_split()

    ### automatic evaluator. takes dataset name as input
    evaluator = Evaluator(name='ogbg-molhiv')

    train_loader = DataLoader(dataset[split_idx["train"]], batch_size=32, shuffle=True, num_workers = 0)
    valid_loader = DataLoader(dataset[split_idx["valid"]], batch_size=32, shuffle=False, num_workers = 0)
    test_loader = DataLoader(dataset[split_idx["test"]], batch_size=32, shuffle=False, num_workers = 0)

    num_features = dataset[0].x.shape[1]
    edge_attr = dataset[0].edge_attr.shape[1]
    model = GINENet(num_features, dim_h=64, edge_attr=edge_attr).to(device)

    optimizer = optim.Adam(model.parameters(), lr=0.001)

    valid_curve = []
    test_curve = []
    train_curve = []

    for epoch in range(1, 101):
        print("=====Epoch {}".format(epoch))
        print('Training...')
        train(model, device, train_loader, optimizer, dataset.task_type)

        print('Evaluating...')
        train_perf = eval(model, device, train_loader, evaluator)
        valid_perf = eval(model, device, valid_loader, evaluator)
        test_perf = eval(model, device, test_loader, evaluator)

        print({'Train': train_perf, 'Validation': valid_perf, 'Test': test_perf})

        train_curve.append(train_perf[dataset.eval_metric])
        valid_curve.append(valid_perf[dataset.eval_metric])
        test_curve.append(test_perf[dataset.eval_metric])

    if 'classification' in dataset.task_type:
        best_val_epoch = np.argmax(np.array(valid_curve))
        best_train = max(train_curve)
    else:
        best_val_epoch = np.argmin(np.array(valid_curve))
        best_train = min(train_curve)

    print('Finished training!')
    print('Best validation score: {}'.format(valid_curve[best_val_epoch]))
    print('Test score: {}'.format(test_curve[best_val_epoch]))

    if not file_name == '':
        torch.save({'Val': valid_curve[best_val_epoch], 'Test': test_curve[best_val_epoch], 'Train': train_curve[best_val_epoch], 'BestTrain': best_train}, file_name)


In [17]:
if __name__ == "__main__":
    main()

=====Epoch 1
Training...


Iteration: 100%|██████████| 1029/1029 [00:22<00:00, 45.99it/s]


Evaluating...


Iteration: 100%|██████████| 129/129 [00:01<00:00, 113.81it/s]


{'Train': {'rocauc': 0.552284284008328}, 'Validation': {'rocauc': 0.4723967519106408}, 'Test': {'rocauc': 0.45092605110179806}}
=====Epoch 2
Training...


...

{'Train': {'rocauc': 0.9477054894724778}, 'Validation': {'rocauc': 0.7625018371546148}, 'Test': {'rocauc': 0.715239769018328}}
=====Epoch 100
Training...


Iteration: 100%|██████████| 1029/1029 [00:19<00:00, 53.12it/s]


Evaluating...


Iteration: 100%|██████████| 129/129 [00:00<00:00, 198.43it/s]


{'Train': {'rocauc': 0.9473687063591623}, 'Validation': {'rocauc': 0.7848140799529689}, 'Test': {'rocauc': 0.6956661967206783}}
Finished training!
Best validation score: 0.8280790711346265
Test score: 0.7405733984820101


In [ ]:
import pyg_format.transformation as T
import copy

chemprop = copy.deepcopy(graph_dataset)
chemprop_vn = T.apply_vn(graph_dataset)
chemprop_deg = T.centrality(graph_dataset, centrality_measure='degree')
chemprop_clo = T.centrality(graph_dataset, centrality_measure='closeness')
chemprop_bet = T.centrality(graph_dataset, centrality_measure='betweenness')
chemprop_eig = T.centrality(graph_dataset, centrality_measure='eigenvector')
chemprop_DE = T.distance_encoding(graph_dataset, method='node_augmentation')
chemprop_GE = T.graph_encoding(graph_dataset, k=2)
chemprop_SE = T.subgraph_extraction(graph_dataset, radius=2)
chemprop_exN = T.extra_node(graph_dataset)